In [ ]:
# !cd ../input/pretrainedmodels
# !python setup.py install
! pip install ../input/mlcollection/ml_collections-0.1.0-py3-none-any.whl

In [ ]:
package_path = '../input/pytorch-image-models/pytorch-image-models-master' #'../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0'
import sys; sys.path.append(package_path)
sys.path.insert(0, '../input/')
sys.path.insert(0, '../input/ibnnet')
sys.path.insert(0, '../input/cass-net')
sys.path.insert(0, '../input/pretrainedmodels')
sys.path.insert(0, '../input/local-libs')



In [ ]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm
from fastai.vision.all import *
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from  torch.cuda.amp import autocast, GradScaler

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
import pydicom
import timm #from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom
from sklearn.metrics import log_loss


In [ ]:
CFG = {
    'fold_num': 5,
    'seed': 719,
    'model_arch': 'resnext101_ibn_a',#'resnext101',#'resnext101_ibn_a',#'se50_softlabel',#'resnext101_ibn_a',#'tf_efficientnet_b4_ns',
    'model_arch_eff':'tf_efficientnet_b4_ns',
    'img_size': 512,
    'epochs': 10,
    'train_bs': 32,
    'valid_bs': 32,
    'lr': 1e-4,
    'num_workers': 4,
    'accum_iter': 1, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'tta': 4,

}

ckpt_paths = ['../input/tf-efficientnet-b4-ns-fold-0-8-089533/tf_efficientnet_b4_ns_fold_0_4_0.89299',
              '../input/tf-efficientnet-b4-ns-fold-0-8-089533/tf_efficientnet_b4_ns_fold_0_6_0.89065',
             '../input/tf-efficientnet-b4-ns-fold-0-8-089533/tf_efficientnet_b4_ns_fold_0_8_0.89533',
              '../input/tf-efficientnet-b4-ns-fold-0-8-089533/tf_efficientnet_b4_ns_fold_0_9_0.88995'
             ]

# version 4 tta=3
ckpt_paths = [
             '../input/resnext101-ibn-a-01-22-10-05-bit-ls015-896/resnext101_ibn_a_fold_0_6_0.88435',
             '../input/resnext101-ibn-a-01-22-10-05-bit-ls015-896/resnext101_ibn_a_fold_0_7_0.89299',
             '../input/resnext101-ibn-a-01-22-10-05-bit-ls015-896/resnext101_ibn_a_fold_0_8_0.88995',
             '../input/resnext101-ibn-a-01-22-10-05-bit-ls015-896/resnext101_ibn_a_fold_0_9_0.89603']
ckpt_weights =  [1,1,1,1]

ckpt_dir = '../input/resnext101-ibn-a-01-26-01-27-bit-ema-5folds'
ckpt_paths = [os.path.join(ckpt_dir, sub_path) for sub_path in os.listdir(ckpt_dir) if not ckpt_dir.endswith('.txt')]

ckpt_paths = ['../input/resnext101-ibn-a-01-26-01-27-bit-ema-5folds/resnext101_ibn_a_fold_0_9_0.89439',
              '../input/resnext101-ibn-a-01-26-01-27-bit-ema-5folds/resnext101_ibn_a_fold_0_19_0.89393',
              '../input/resnext101-ibn-a-01-26-01-27-bit-ema-5folds/resnext101_ibn_a_fold_1_9_0.89603',
              '../input/resnext101-ibn-a-01-26-01-27-bit-ema-5folds/resnext101_ibn_a_fold_1_8_0.89486',
              '../input/resnext101-ibn-a-01-26-01-27-bit-ema-5folds/resnext101_ibn_a_fold_2_9_0.88759',
              '../input/resnext101-ibn-a-01-26-01-27-bit-ema-5folds/resnext101_ibn_a_fold_2_17_0.88619',
              '../input/resnext101-ibn-a-01-26-01-27-bit-ema-5folds/resnext101_ibn_a_fold_3_16_0.89928',
              '../input/resnext101-ibn-a-01-26-01-27-bit-ema-5folds/resnext101_ibn_a_fold_3_19_0.89904',
              '../input/resnext101-ibn-a-01-26-01-27-bit-ema-5folds/resnext101_ibn_a_fold_4_18_0.89857',
              '../input/resnext101-ibn-a-01-26-01-27-bit-ema-5folds/resnext101_ibn_a_fold_4_9_0.89787',
             ]


#ckpt_dir = '../input/resnext101ibn0201select'
#ckpt_paths = [os.path.join(ckpt_dir, fn) for fn in os.listdir(ckpt_dir)]


#ckpt_dir = '../input/alldata-ema-resnext101-ibn-a'
#ckpt_paths = [os.path.join(ckpt_dir, fn) for fn in os.listdir(ckpt_dir)]


ckpt_paths = [
    
              #'../input/seresnex101ibn-timm/checkpoint-21_f0_90.16.pth.tar',
              #'../input/seresnex101ibn-timm/checkpoint-18_f1_9014.pth.tar',
              #'../input/seresnex101ibn-timm/checkpoint-30_f2_90.04.pth.tar',
              #'../input/seresnex101ibn-timm/checkpoint-30_f3_88.52.pth.tar',
              #'../input/seresnex101ibn-timm/checkpoint-16_f4_89.57.pth.tar',
                
             '../input/vit-convert-tta-seed719/vit__1920-fold0_checkpoint-17.pth',
              '../input/vit-convert-tta-seed719/vit__1920-fold1_checkpoint-14.pth',
              '../input/vit-convert-tta-seed719/vit__1920-fold2_checkpoint-16.pth',
              '../input/vit-convert-tta-seed719/vit__1920-fold3_checkpoint-8.pth',
              '../input/vit-convert-tta-seed719/vit__1920-fold4_checkpoint-9.pth',
    
             '../input/alldata-ema-resnext101-ibn-a/resnext101_ibn_a_fold_0_11_0.89603',
             '../input/alldata-ema-resnext101-ibn-a/resnext101_ibn_a_fold_1_10_0.89369',
             '../input/alldata-ema-resnext101-ibn-a/resnext101_ibn_a_fold_2_13_0.88899',
             '../input/alldata-ema-resnext101-ibn-a/resnext101_ibn_a_fold_3_12_0.89834',
             '../input/alldata-ema-resnext101-ibn-a/resnext101_ibn_a_fold_4_11_0.89741',

             '../input/alldata-ema-resnext101-32x4d-swsl-adam-02-05-17-28/resnext101_32x4d_swsl_fold_0_12_0.89369',
              '../input/alldata-ema-resnext101-32x4d-swsl-adam-02-05-17-28/resnext101_32x4d_swsl_fold_1_12_0.89416',
              '../input/alldata-ema-resnext101-32x4d-swsl-adam-02-05-17-28/resnext101_32x4d_swsl_fold_2_8_0.89203',
              '../input/alldata-ema-resnext101-32x4d-swsl-adam-02-05-17-28/resnext101_32x4d_swsl_fold_3_12_0.90115',
              '../input/alldata-ema-resnext101-32x4d-swsl-adam-02-05-17-28/resnext101_32x4d_swsl_fold_4_14_0.89530',
             
              #'../input/fork-of-cassava-classification-fastai/models/best_model_512_lce_sgdr_0.pth',
              #'../input/fork-of-cassava-classification-fastai/models/best_model_512_lce_sgdr_1.pth',
              #'../input/fork-of-cassava-classification-fastai/models/best_model_512_lce_sgdr_2.pth',
              #'../input/fork-of-cassava-classification-fastai/models/best_model_512_lce_sgdr_3.pth',
              #'../input/fork-of-cassava-classification-fastai/models/best_model_512_lce_sgdr_4.pth',
            
             
              
             ]
model_names = [  
               # 'se_resnet101_ibn_a',
               # 'se_resnet101_ibn_a',
               # 'se_resnet101_ibn_a',
               # 'se_resnet101_ibn_a',
               # 'se_resnet101_ibn_a',
                
               'vit',
               'vit',
               'vit',
               'vit',
               'vit',
              'resnext101_ibn_a', 
               'resnext101_ibn_a',
              'resnext101_ibn_a',
              'resnext101_ibn_a',
              'resnext101_ibn_a',
               'resnext101',
               'resnext101',
               'resnext101',
               'resnext101',
               'resnext101',
               #'b4',
               #'b4',
               #'b4',
               #'b4',
               #'b4',
              
]
assert len(model_names) == len(ckpt_paths)

print(len(ckpt_paths))

ckpt_weights =  [1 for i in range(len(ckpt_paths))]
print(ckpt_paths)

model_wts={}
#wts={'w0': 0.6465801475470692, 'w1': 0.6582084173812914, 'w2': 0.16946139705016672}


#model_wts['b4']=0.18833718
model_wts['vit']=0.6582084173812914
model_wts['resnext101']=0.16946139705016672
model_wts['resnext101_ibn_a']=0.6465801475470692
#[0.27363746 0.10715301 0.42271631 0.18833718]
# ckpt_paths = [
#              '../input/soft-label-se50-01-25-18-37-8949/se50_fold_0_16_0.89042',
#              '../input/soft-label-se50-01-25-18-37-8949/se50_fold_0_19_0.89276',
#              '../input/soft-label-se50-01-25-18-37-8949/se50_fold_0_29_0.89322',
#              '../input/soft-label-se50-01-25-18-37-8949/se50_fold_0_18_0.89486']
# ckpt_weights =  [1,1,1,1]


# # version 5 tta=4
# ckpt_paths = ['../input/resnext101-ibn-a-01-22-10-05-bit-ls015-896/resnext101_ibn_a_fold_0_6_0.88435',
#              '../input/resnext101-ibn-a-01-22-10-05-bit-ls015-896/resnext101_ibn_a_fold_0_7_0.89299',
#              '../input/resnext101-ibn-a-01-22-10-05-bit-ls015-896/resnext101_ibn_a_fold_0_8_0.88995',
#              '../input/resnext101-ibn-a-01-22-10-05-bit-ls015-896/resnext101_ibn_a_fold_0_9_0.89603']
# ckpt_weights =  [0.2,0.25,0.2,0.35]

# # version 6
# ckpt_paths = ['../input/resnext101-ibn-a-01-22-01-13-bit-ls035-896/resnext101_ibn_a_fold_0_6_0.88972',
#              '../input/resnext101-ibn-a-01-22-01-13-bit-ls035-896/resnext101_ibn_a_fold_0_7_0.89393',
#              '../input/resnext101-ibn-a-01-22-01-13-bit-ls035-896/resnext101_ibn_a_fold_0_8_0.89136',
#              '../input/resnext101-ibn-a-01-22-01-13-bit-ls035-896/resnext101_ibn_a_fold_0_9_0.89650']

# ckpt_weights =  [1,1,1,1]

# #version 8
# ckpt_paths = ['../input/resnext101-ibn-a-01-22-01-13-bit-ls035-896/resnext101_ibn_a_fold_0_6_0.88972',
#              '../input/resnext101-ibn-a-01-22-01-13-bit-ls035-896/resnext101_ibn_a_fold_0_7_0.89393',
#              '../input/resnext101-ibn-a-01-22-01-13-bit-ls035-896/resnext101_ibn_a_fold_0_8_0.89136',
#              '../input/resnext101-ibn-a-01-22-01-13-bit-ls035-896/resnext101_ibn_a_fold_0_9_0.89650']

# ckpt_weights =  [0.2, 0.25, 0.2, 0.35]




In [ ]:
len(ckpt_weights)

In [ ]:
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train.head()

In [ ]:
train.label.value_counts()

> We could do stratified validation split in each fold to make each fold's train and validation set looks like the whole train set in target distributions.

In [ ]:
submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
submission.head()

# Helper Functions

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

img = get_img('../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
plt.imshow(img)
plt.show()

# Dataset

In [ ]:
class CassavaDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['label']
          
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        
        img  = get_img(path)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        # do label smoothing
        if self.output_label == True:
            return img, target
        else:
            return img

# Define Train\Validation Image Augmentations

In [ ]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

def get_train_transforms():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)
  
        
def get_valid_transforms():

    return Compose([
            CenterCrop(CFG['img_size'], CFG['img_size'], p=1.),
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

def get_inference_transforms():
    return Compose([
        RandomResizedCrop(CFG['img_size'], CFG['img_size']),
        Transpose(p=0.5),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.2),
        RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0),
    ], p=1.)


# Model

In [ ]:
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
from ibnnet import *
class IBNResnextCassava(nn.Module):
    def __init__(self, arch='resnext101_ibn_a', n_class=5, pre=False):
        super().__init__()
        #m = torch.hub.load('XingangPan/IBN-Net', arch, pretrained=pre) #'resnext101_ibn_a'
        m = resnext101_ibn_a()
        self.enc = nn.Sequential(*list(m.children())[:-2])
        nc = list(m.children())[-1].in_features
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(2048, n_class)
        )

    def forward(self,  x):
        x = self.enc(x)
        x = self.head(x)

        return x

In [ ]:
class MishFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return x * torch.tanh(F.softplus(x))   # x * tanh(ln(1 + exp(x)))

    @staticmethod
    def backward(ctx, grad_output):
        x = ctx.saved_variables[0]
        sigmoid = torch.sigmoid(x)
        tanh_sp = torch.tanh(F.softplus(x)) 
        return grad_output * (tanh_sp + x * sigmoid * (1 - tanh_sp * tanh_sp))

class Mish(nn.Module):
    def forward(self, x):
        return MishFunction.apply(x)

def to_Mish(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, Mish())
        else:
            to_Mish(child)
            
def _resnext(url, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
#     state_dict = load_state_dict_from_url(url, progress=progress)
#     model.load_state_dict(state_dict)
    return model


In [ ]:
from torchvision.models.resnet import ResNet, Bottleneck


semi_weakly_supervised_model_urls = {
    'resnet18':         'https://dl.fbaipublicfiles.com/semiweaksupervision/model_files/semi_weakly_supervised_resnet18-118f1556.pth',
    'resnet50':         'https://dl.fbaipublicfiles.com/semiweaksupervision/model_files/semi_weakly_supervised_resnet50-16a12f1b.pth',    
    'resnext50_32x4d':  'https://dl.fbaipublicfiles.com/semiweaksupervision/model_files/semi_weakly_supervised_resnext50_32x4-72679e44.pth',
    'resnext101_32x4d': 'https://dl.fbaipublicfiles.com/semiweaksupervision/model_files/semi_weakly_supervised_resnext101_32x4-3f87e46b.pth',
    'resnext101_32x8d': 'https://dl.fbaipublicfiles.com/semiweaksupervision/model_files/semi_weakly_supervised_resnext101_32x8-b4712904.pth',
    'resnext101_32x16d':'https://dl.fbaipublicfiles.com/semiweaksupervision/model_files/semi_weakly_supervised_resnext101_32x16-f3559a9c.pth',
}


class ResnextCassava(nn.Module):
    def __init__(self, arch='resnext101_32x4d', n_class=5, pretrained=False):
        super().__init__()

        m = _resnext(semi_weakly_supervised_model_urls[arch], Bottleneck, [3, 4, 23, 3], False, 
                progress=False,groups=32,width_per_group=4)
        
        self.enc = nn.Sequential(*list(m.children())[:-2])       
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.logit = nn.Linear(2048,n_class)

        
    def forward(self,  x):
        x = self.enc(x)
        x = (self.avg_pool(x) + self.max_pool(x)).squeeze(-1).squeeze(-1)
        x = self.logit(x)
        return x



# class ResnextCassava(nn.Module):
#     def __init__(self, arch='resnext50_32x4d', n_class=5, pretrained=False):
#         super().__init__()
#         m = torch.hub.load('facebookresearch/semi-supervised-ImageNet1K-models', 'resnext101_32x4d_swsl')

#         self.enc = nn.Sequential(*list(m.children())[:-2])       
#         nc = list(m.children())[-1].in_features
#         self.avg_pool = nn.AdaptiveAvgPool2d(1)
#         self.max_pool = nn.AdaptiveMaxPool2d(1)
        
#         self.logit = nn.Linear(2048,n_class)
#     def forward(self,  x):
#         x = self.enc(x)
#         #print('enc:', x.shape)
#         x = (self.avg_pool(x) + self.max_pool(x)).squeeze(-1).squeeze(-1)
#         x = self.logit(x)
#         return x

In [ ]:
class timm_class(nn.Module):
    def __init__(self, model, n=5 ):
        super().__init__()
        #m = torch.hub.load('facebookresearch/semi-supervised-ImageNet1K-models', arch,pre)
        #m = _resnext(semi_supervised_model_urls[arch], Bottleneck, [3, 4, 6, 3], False, 
        #        progress=False,groups=32,width_per_group=4)
        #self.enc = nn.Sequential(*list(m.children())[:-2])       
        
        self.effnet =  nn.Sequential(*list(model.children())[:-2])      
        #nc = list(self.effnet.children())[-2].in_features
        nc = self.effnet[-3].out_channels
        #print(nc)
        self.dropout = nn.Dropout(0.2)
        #self.out = nn.Linear(1536, num_classes)
        #self.arcmargin=ArcMarginProduct(in_features=512,out_features=n).to(device)
        self.head = nn.Sequential(AdaptiveConcatPool2d(),Flatten(),nn.GroupNorm(32,2*nc)
                                  ,nn.Linear(2*nc,512),
                            Mish(),nn.GroupNorm(16,512), nn.Dropout(0.4), nn.Linear(512,n))
        
        #self.head = nn.Sequential(AdaptiveConcatPool2d(),Flatten(),
        #                          nn.BatchNorm1d(2*nc) ,nn.Linear(2*nc,512),
        #                    Mish(),nn.BatchNorm1d(512), nn.Dropout(0.4),nn.Linear(512,n))
        
        
        
        #layers=[AdaptiveConcatPool2d(),Flatten()]+bn_drop_lin(nc*2, 512, True, 0.5, Mish()) + \
        #    bn_drop_lin(512, n, True, 0.3) #nn.BatchNorm1d(512)
        #self.head = nn.Sequential(*layers)
        
    def forward(self,  x):
        #print(len(x))
        '''
        shape = x[0].shape
        n = len(x)
        x = torch.stack(x,1).view(-1,shape[1],shape[2],shape[3]).cpu()
        # filter white features
        #x: bs*N x 3 x 128 x 128
        #l=[ get_emptiness(empty)  for empty in  x ]
        #print(l)
        #print(x[0])
        #print(x.size())
        #x1=[empty  for empty in  x if get_emptiness(empty)<0.80]
        print(len(x))
        #n=len(x)
        #x = self.enc(torch.stack(x))
        '''
        #x = self.effnet.extract_features(x)
        #print('shape',x.shape)
        x=self.effnet(x)
        
        #x = self.enc(x)
        #x: bs*N x C x 4 x 4
        shape = x.shape
        #print('shape',shape)
        #concatenate the output for tiles into a single map
        #x = x.view(-1,n,shape[1],shape[2],shape[3]).permute(0,2,1,3,4).contiguous()\
        #  .view(-1,shape[1],shape[2]*n,shape[3])
        #x: bs x C x N*4 x 4
        #print(x.size())
        #print('x',x.size())
        x = self.head(x)
        #print('out',x.size())
        #x: bs x n
        return x

In [ ]:
from timm import create_model
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        
        self.model = create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        '''
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            #nn.Linear(n_features, hidden_size,bias=True), nn.ELU(),
            nn.Linear(n_features, n_class, bias=True)
        )
        '''
    def forward(self, x):
        x = self.model(x)
        return x

class IBN_CUSTOM1(nn.Module):
    def __init__(self, arch='resnext101_ibn_a', n_class=5, pretrained=True):
        super().__init__()
        m = se_resnet101_ibn_a(pretrained=pretrained)
        self.enc = nn.Sequential(*list(m.children())[:-2])
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(2048, n_class)
        )

    def forward(self,  x):
        x = self.enc(x)
        x = self.head(x)

        return x
    

def get_model(model_name, device='cuda'):
    if 'efficientnet' in model_name:
#         from timm import create_model
#         #model = CassvaImgClassifier(model_name, train.label.nunique(), pretrained=False)
#         model = create_model(
#             model_name,
#             num_classes=5) #bs =16
        model = CassvaImgClassifier(model_name, 5, pretrained=False).to(device)

    elif model_name == 'ibn50a':
        from cassnet import resnet50_ibn_a
        model = resnet50_ibn_a(pretrained=False).to(device)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, train.label.nunique())
    elif model_name == 'ibnse50a':
        from cassnet import se_resnet50_ibn_a
        model = se_resnet50_ibn_a(pretrained=False).to(device)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, train.label.nunique())
    
    elif model_name == 'resnext101_ibn_a':
        model = IBNResnextCassava(arch=model_name)
    elif model_name == 'se50_softlabel':
        from net_soft_label import se_resnext50_32x4d
        model = se_resnext50_32x4d(pretrained=None).to(device)
    elif model_name == 'resnext101':
        model = ResnextCassava(pretrained=False).to(device)
    elif model_name == 'b4':
        b4_model = timm.create_model('tf_efficientnet_b4_ns', pretrained=False)
        
        model=timm_class(b4_model,n=5).to(device)    
    elif model_name == 'vit':
        from vitnet import VisionTransformer
        import vitnet.configs as configs
        CONFIGS = {
            'ViT-B_16': configs.get_b16_config(),
            'ViT-B_32': configs.get_b32_config(),
            'ViT-L_16': configs.get_l16_config(),
            'ViT-L_32': configs.get_l32_config(),
            'ViT-H_14': configs.get_h14_config(),
            'testing': configs.get_testing(),
        }
        config = CONFIGS["ViT-B_16"]
        model = VisionTransformer(config, CFG['img_size'], zero_head=True, num_classes=5).to(device)
    
    elif model_name == 'se_resnet101_ibn_a':
        model = IBN_CUSTOM1(pretrained=False).to(device) #bs=18

        
    
    
    model = model.to(device)
    
    #model = nn.DataParallel(model.to(device))
    return model
    

# Main Loop

In [ ]:
! pip install ../input/mlcollection/ml_collections-0.1.0-py3-none-any.whl

In [ ]:
class AdaptiveConcatPool2d(nn.Module):
    
    "Layer that concats `AdaptiveAvgPool2d` and `AdaptiveMaxPool2d`"
    def __init__(self, size=None):
        super().__init__()
        self.size = size or 1
        self.ap = nn.AdaptiveAvgPool2d(self.size)
        self.mp = nn.AdaptiveMaxPool2d(self.size)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [ ]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        image_preds = model(imgs)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    #image_preds_min=np.min(image_preds_all,axis=-1)
    #image_preds_max=np.max(image_preds_all,axis=-1)
    #image_preds_norm=(image_preds_all-image_preds_min[:,np.newaxis])/ (image_preds_max[:,np.newaxis]-image_preds_min[:,np.newaxis])
 
    return image_preds_all

In [ ]:
if __name__ == '__main__':
     # for training only, need nightly build pytorch
    VALID = False#False
    test_num = len(os.listdir('../input/cassava-leaf-disease-classification/test_images'))    
    print('test_num:', test_num)

    seed_everything(CFG['seed'])
    
    #folds = StratifiedKFold(n_splits=CFG['fold_num']).split(np.arange(train.shape[0]), train.label.values)
    folds = StratifiedKFold(n_splits=CFG['fold_num'], shuffle=True, random_state=CFG['seed']).split(np.arange(train.shape[0]), train.label.values)

    for fold, (trn_idx, val_idx) in enumerate(folds):
        # we'll train fold 0 first
        if fold > 0:
            break 

        print('Inference fold {} started'.format(fold))

        valid_ = train.loc[val_idx,:].reset_index(drop=True)
        valid_ds = CassavaDataset(valid_, '../input/cassava-leaf-disease-classification/train_images/', transforms=get_inference_transforms(), output_label=False)
        
        test = pd.DataFrame()
        test['image_id'] = list(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))
        test_ds = CassavaDataset(test, '../input/cassava-leaf-disease-classification/test_images/', transforms=get_inference_transforms(), output_label=False)
        
        val_loader = torch.utils.data.DataLoader(
            valid_ds, 
            batch_size=CFG['valid_bs'],
            num_workers=CFG['num_workers'],
            shuffle=False,
            pin_memory=False,
        )
        
        tst_loader = torch.utils.data.DataLoader(
            test_ds, 
            batch_size=CFG['valid_bs'],
            num_workers=CFG['num_workers'],
            shuffle=False,
            pin_memory=False,
        )

        device = torch.device(CFG['device'])

        val_preds = []
        tst_preds = []
        
        for i in range(len(ckpt_paths)):   
            ckpt_path = ckpt_paths[i]
            model = get_model(model_names[i], device)
            
            if model_names[i] == 'se_resnet101_ibn_a':
                model.load_state_dict(torch.load(ckpt_path)['state_dict'])
            else:
                model.load_state_dict(torch.load(ckpt_path,map_location=device))
            
            with torch.no_grad():
                for _ in range(CFG['tta']):
                    if VALID:
                        if test_num == 1:
                            val_preds += [ckpt_weights[i]/sum(ckpt_weights)/CFG['tta']*inference_one_epoch(model, val_loader, device)]
                    
                    #tst_preds += [ckpt_weights[i]/sum(ckpt_weights)/CFG['tta']*
                    #              inference_one_epoch(model, tst_loader, device)] model_wts[model_names[i]]
                    tst_preds += [ 
                                  inference_one_epoch(model, tst_loader, device)*model_wts[model_names[i]]]
                    #print(len(tst_preds))
            del model
            torch.cuda.empty_cache()
        if VALID:
            if test_num == 1:
                val_preds = np.mean(val_preds, axis=0) 
        tst_preds = np.mean(tst_preds, axis=0) *3  #multiply by number of models if weighted
        
        if VALID:
            if test_num == 1:
                print('fold {} validation loss = {:.5f}'.format(fold, log_loss(valid_.label.values, val_preds)))
            print('fold {} validation accuracy = {:.5f}'.format(fold, (valid_.label.values==np.argmax(val_preds, axis=1)).mean()))


            from sklearn.metrics import confusion_matrix
            cm = confusion_matrix(valid_.label.values, np.argmax(val_preds, axis=1), labels=[0, 1, 2, 3, 4])
            print('cm')
            print(cm)
            import seaborn as sns
            sns.set()
            f, ax=plt.subplots()
            sns.heatmap(cm,annot=True,ax=ax) #画热力图

            ax.set_title('confusion matrix') #标题
            ax.set_xlabel('predict') #x轴
            ax.set_ylabel('true') #y轴

#         del model
#         torch.cuda.empty_cache()

In [ ]:
tst_preds

In [ ]:
test['label'] = np.argmax(tst_preds, axis=1)
test.head()

In [ ]:
test.to_csv('submission.csv', index=False)

# Train part is here: https://www.kaggle.com/khyeh0719/pytorch-efficientnet-baseline-train-amp-aug

In [ ]:
len(tst_preds)